In [9]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from sklearn.cluster import AgglomerativeClustering
import dash
from dash import dcc
from dash import html

In [20]:
ticker = "BNB-USD"
rolling_wave_length = 20
num_clusters = 4
period = '8d'
interval = '1h'

In [21]:
df = yf.download(ticker, period = period, interval = interval)
len(df)

[*********************100%***********************]  1 of 1 completed


192

In [22]:
#creating function
def calculate_support_resistance(df, rolling_wave_length, num_clusters):
    date = df.index
    df.reset_index(inplace=True)
    
    max_waves_temp = df.High.rolling(rolling_wave_length).max().rename('waves')
    min_waves_temp = df.Low.rolling(rolling_wave_length).min().rename('waves')
   
    max_waves = pd.concat([max_waves_temp, pd.Series(np.zeros(len(max_waves_temp)) + 1)], axis=1)
    min_waves = pd.concat([min_waves_temp, pd.Series(np.zeros(len(min_waves_temp)) + -1)], axis=1)
    max_waves.drop_duplicates('waves', inplace=True)
    min_waves.drop_duplicates('waves', inplace=True)
    
    waves = pd.concat([max_waves, min_waves]).sort_index()
    waves = waves[waves[0] != waves[0].shift()].dropna()
    
    x = np.concatenate((waves.waves.values.reshape(-1, 1),
                        (np.zeros(len(waves)) + 1).reshape(-1, 1)), axis=1)
    
    cluster = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
    cluster.fit_predict(x)
    waves['clusters'] = cluster.labels_
    waves2 = waves.loc[waves.groupby('clusters')['waves'].idxmax()]
    df.index = date
    waves2.waves.drop_duplicates(keep='first', inplace=True)
    
    return waves2.reset_index().waves

In [23]:
support_resistance_levels = calculate_support_resistance(df, rolling_wave_length, num_clusters)

In [24]:
#creating a plot
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.06, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.3, 0.7])

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = "Market data"), row = 1, col = 1)

fig.update_xaxes(
    rangeslider_visible = False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=3, label="3d", step="day", stepmode="backward"),
            dict(count=7, label="7d", step="day", stepmode="backward"),
            dict(count=30, label="30d", step="day", stepmode="backward"),
            dict(step="all")])))

i = 0
for level in support_resistance_levels.to_list():
    fig.add_hline(y=level, line_width=1, line_dash="dash", row=1, col=1)
    i += 1

colors = []

for i in range(len(df.Close)):
    if i != 0:
        if df.Close[i] > df.Close[i-1]:
            colors.append('green')
        else:
            colors.append('red')
    else:
        colors.append('red')

fig.add_trace(go.Bar(x=df.index, y=df['Volume'], showlegend=False, marker=dict(color=colors)), row=2, col=1)

fig.update_traces(name= 'Volume', selector=dict(type='bar'))

text = f'{ticker} Chart'

fig.update_layout(
    title=go.layout.Title(
        text=text,
        xref="paper",
        x=0))


fig.show()